# Training models

## Models

### Definition

The representation learnt from data during training is called a **model**. It defines the relationship between inputs and outputs, and thus produces results from data. Most (but not all) ML systems are model-based.

[![Extract from the book Hands-on Machine Learning with Scikit-Learn & TensorFlow by A. Géron](images/instance_model_learning.png)](https://github.com/ageron/handson-ml2)

### Hypothesis function

- $\pmb{\theta}$ (sometime noted $\pmb{\omega}$): model's internal parameters vector.
- $h_\theta()$: model's prediction function (*hypothesis function*), using the model parameters $\pmb{\theta}$ to define the relationship between features and labels.
- $y'^{(i)}$ (sometimes noted $\hat{y}^{(i)}$): hypothesis function output (model prediction).

$$y'^{(i)} = h_\theta(\pmb{x}^{(i)})$$ 

### Multiclass classification

* $\pmb{y}^{(i)}$ et $\pmb{y}'^{(i)}$ are vectors with as many elements as the number of predicted classes $K$.
* $\pmb{y}^{(i)}$ (the *ground truth*) is a **binary vector** of $K$ values. $y^{(i)}_k$ is equal to 1 if the $i$th sample's class corresponds to $k$, 0 otherwise.
* $\pmb{y}'^{(i)}$ is a **probability vector** of $K$ values, computed by the model. $y'^{(i)}_k$ represents the probability that the $i$th sample belongs to class $k$.

$$\pmb{y}^{(i)} = \begin{pmatrix}
       \ y^{(i)}_1 \\
       \ y^{(i)}_2 \\
       \ \vdots \\
       \ y^{(i)}_K
     \end{pmatrix} \in \pmb{R}^K
\;\;\;
\pmb{y}'^{(i)} = \begin{pmatrix}
       \ y'^{(i)}_1 \\
       \ y'^{(i)}_2 \\
       \ \vdots \\
       \ y'^{(i)}_K
     \end{pmatrix} \in \pmb{R}^K$$

### Model lifecycle

There are two (repeatable) phases:

- **Training**: using training input samples, the model learns to find a relationship between features and labels.
- **Inference**: the trained model is used to make predictions.

### Hyperparameters

- Many ML models also have user-defined properties called **hyperparameters**.
  - Examples: maximum depth of a decision tree, number oy layers of a neural network...
- Contrary to internal parameters, they are not automatically updated during training.
- The hyperparameters directly affect the model's performance and must be tweaked during the training and tuning steps.

## Loss

### Definition

- $\mathcal{L_{\pmb{X, y}}(\pmb{\theta})}$, sometimes noted $\mathcal{J_{\pmb{X, y}}(\pmb{\theta})}$: **loss** (or **cost**) function that quantifies the difference, often called **error**, between expected results (called *ground truth*) and actual results computed by the model.
- During model training, the input dataset $\pmb{X}$ and the expected results $\pmb{y}$ are treated as constants. The loss depends solely on the model parameters $\pmb{\theta}$. To simplify notations, the loss function will be written $\mathcal{L(\pmb{\theta})}$.
- Different loss functions exist. The choice depends on the learning type. See [Losses](./losses) for details.

## Optimization algorithm

### Definition

- Used during the training phase.
- Objective: find the set of model parameters $\pmb{\theta}^{*}$ that minimizes the loss.
- For each learning type, several algorithms of various complexity exist.

[![Optimization: linear regression](images/LossSideBySide.png)](https://developers.google.com/machine-learning/crash-course/reducing-loss/an-iterative-approach)

## Gradient descent

### An iterative approach

The model's parameters are iteratively updated until an optimum is reached.

[![Iterative approach](images/GradientDescentDiagram.png)](https://developers.google.com/machine-learning/crash-course/descending-into-ml/training-and-loss)

### The gradient descent algorithm

- Used in several ML models, including neural networks.
- General idea: converging to a loss function's minimum by updating model parameters in small steps, in the **opposite direction** of the loss function **gradient**.

### The notion of gradient

- Expresses the variation of a function relative to the variation of its parameters.
- Vector containing partial derivatives of the function *w.r.t.* each of its $P$ parameters.

$$\nabla_{\theta}\mathcal{L}(\boldsymbol{\pmb{\theta}}) = \begin{pmatrix}
       \ \frac{\partial}{\partial \theta_1} \mathcal{L}(\boldsymbol{\theta}) \\
       \ \frac{\partial}{\partial \theta_2} \mathcal{L}(\boldsymbol{\theta}) \\
       \ \vdots \\
       \ \frac{\partial}{\partial \theta_P} \mathcal{L}(\boldsymbol{\theta})
     \end{pmatrix}$$

### 1D gradient descent (one parameter)

![Gradient Descent](images/gradient_descent_1parameter.png)

### 2D gradient (two parameters)

![Tangent Space](images/tangent_space.png)

### 2D gradient descent

![Gradient Descent](images/gradient_descent_2parameters.gif)

## Gradient descent types

### Batch Gradient Descent

The gradient is computed on the whole dataset before model parameters are updated.

- Advantages: simple and safe (always converges in the right direction).
- Drawback: can become slow and even untractable with a big dataset.

### Stochastic Gradient Descent (SGD)

The gradient is computed on only one randomly chosen sample whole dataset before parameters are updated.

- Advantages:
  - Very fast.
  - Enables learning from each new sample (*online learning*).
- Drawback:
  - Convergence is not guaranteed.
  - No vectorization of computations.

### Mini-Batch SGD

The gradient is computed on a small set of samples, called a *batch*, before parameters are updated.

- Combines the advantages of batch and stochastic GD.
- Default method for many ML libraries.
- The mini-batch size varies between 10 and 1000 samples, depending of the dataset size.

## Model parameters update

### Learning rate

$\eta$ is the update factor for parameters once gradient is computed, called the **_learning rate_**.

It has a direct impact on the "speed" of the gradient descent.

$$\pmb{\theta_{next}} = \pmb{\theta} - \eta\nabla_{\boldsymbol{\theta}}\mathcal{L}(\boldsymbol{\theta})$$

### Importance of learning rate

![Learning rate](images/learning_rate.png)

[Interactive exercise](https://developers.google.com/machine-learning/crash-course/fitter/graph)

### The local minima problem

![Local minima](images/local_minima.jpg)

![Gradient Descent](images/gd_ng.jpg)

## Optimization algorithms

### Gradient descent evolution map

[![Gradient Descent evolution map](images/gradient_descent_evolution_map.png)](https://towardsdatascience.com/10-gradient-descent-optimisation-algorithms-86989510b5e9)

### Momentum

Momentum optimization accelerates the descent speed in the direction of the minimum by accumulating previous gradients. It can also escape plateaux faster then plain GD.

[![Momemtum demo](images/gd_momentum_demo.gif)](https://youtu.be/qPKKtvkVAjY)

#### Momentum equations

$$\pmb{m_{k+1}} = \beta_k \pmb{m_k} - \nabla_{\boldsymbol{\theta}}\mathcal{L}(\boldsymbol{\theta_k})$$

$$\pmb{\theta_{k+1}} = \pmb{\theta_k} + \eta_k\pmb{m_{k+1}}$$

$\beta_k \in [0,1]$ is a friction factor that prevents gradients updates from growing too large. A typical value is 0.9.

#### Momentum Vs plain GD

[![Momentum Vs plain GD](images/gd_momentum.png)](https://youtu.be/kVU8zTI-Od0)

### RMSprop

*RMSprop* decays the learning rate differently for each parameter, scaling down the gradient vector along the steepest dimensions. The underlying idea is to adjust the descent direction a bit more towards the global minimum.

$$\pmb{v_{k+1}} = \beta_k \pmb{v_k} + (1-\beta_k) \left(\nabla_{\boldsymbol{\theta}}\mathcal{L}(\boldsymbol{\theta_k})\right)^2$$

$$\pmb{\theta_{k+1}} = \pmb{\theta_k} - \frac{\eta_k}{\sqrt{\pmb{v_{k}}+\epsilon}}\nabla_{\boldsymbol{\theta}}\mathcal{L}(\boldsymbol{\theta_k})$$

$\epsilon$ is a smoothing term to avoid divisions by zero. A typical value is $10^{-10}$.

### Adam and other techniques

*Adam* (*Adaptive Moment Estimation*) combines the ideas of momentum and RMSprop. It is the *de facto* choice nowadays.

Gradient descent optimization is a rich subfield of Machine Learning. Read more in [this article](http://ruder.io/optimizing-gradient-descent/).